In [1]:
!python -V

Python 3.12.10


In [2]:
import pandas as pd
import mlflow

In [3]:
df = pd.read_parquet('./data/yellow_tripdata_2023-03.parquet')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3403766 entries, 0 to 3403765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [4]:
print(len(df))

3403766


In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df = read_dataframe('./data/yellow_tripdata_2023-03.parquet')

print(len(df))

3316216


In [7]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,15.6,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333


In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

import mlflow.sklearn

In [9]:

mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("hw3-experiment")


2025/06/10 19:32:22 INFO mlflow.tracking.fluent: Experiment with name 'hw3-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:C:/Users/ardih/Study/DataTalksClub/MLOps_2025/mlops-zoomcamp-2025/hw3/mlflow_runs/1', creation_time=1749558742783, experiment_id='1', last_update_time=1749558742783, lifecycle_stage='active', name='hw3-experiment', tags={}>

In [10]:
categorical = ['PULocationID', 'DOLocationID']
#numerical = ['trip_distance']

train_dicts = df[categorical].to_dict(orient='records')

train_dicts[:5]

[{'PULocationID': '238', 'DOLocationID': '42'},
 {'PULocationID': '138', 'DOLocationID': '231'},
 {'PULocationID': '140', 'DOLocationID': '186'},
 {'PULocationID': '140', 'DOLocationID': '43'},
 {'PULocationID': '79', 'DOLocationID': '137'}]

In [11]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

print(X_train.shape)
print(y_train.shape)

(3316216, 518)
(3316216,)


In [12]:

# Start an MLflow run
with mlflow.start_run():

    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Log parameters
    mlflow.log_param('intercept', model.intercept_)

    # Make predictions
    y_pred = model.predict(X_train)
    
    # Calculate metrics
    rmse = root_mean_squared_error(y_train, y_pred)

    # Log metrics
    mlflow.log_metric("rmse", rmse)

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="linear_regression_model",
        registered_model_name="linear-regression-model"
    )
    

2025/06/10 19:38:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'linear-regression-model'.
2025/06/10 19:38:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linear-regression-model, version 1


🏃 View run dapper-dove-552 at: http://127.0.0.1:8080/#/experiments/1/runs/684ffdeb50604820ad677f892419b45a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1


Created version '1' of model 'linear-regression-model'.
